# MSDS 462 Demo 6

Training a basic model, then exporting to a tflite model to run on a raspberry pi

In [1]:
import numpy as np
from matplotlib import pyplot as plt

# tensorflow stuff
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow import lite


# importing the mnist dataset
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784')
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from subprocess import run

In [2]:
# preparing the data -- reshaping, creating the one hot array for the softmax etc
in_set = np.array(mnist.data).reshape(70000,28,28,1)
out_set = np.zeros((70000,10)) # empty for the one-hot encoding
out_set[np.arange(70000), np.array(mnist.target, dtype=int)] = 1 # ones for the correct value


# get the testing and training set
in_train, in_test, out_train, out_test = train_test_split(in_set, out_set, train_size=.8)

### Build and train the model

In [4]:
mdl = Sequential() # create the model

mdl.add(Conv2D(1,3, activation='relu', input_shape=[28,28,1]))
mdl.add(MaxPooling2D())
mdl.add(Dropout(.25)) # add a dropout layer
mdl.add(Conv2D(1,3, activation='relu', input_shape=[25,25,1])) # down by 3 from the conv and the pooling
# mdl.add(MaxPooling2D())
# mdl.add(Conv2D(1,3, activation='relu', input_shape=[22,22,1])) # down by 3 from the conv and the pooling mdl.add(MaxPooling2D())
mdl.add(Flatten())
mdl.add(Dense(10, activation='softmax')) # for the 10 values of the one-hot

mdl.compile(loss='CategoricalCrossentropy', optimizer='rmsprop', metrics='Accuracy')
mdl.fit(in_train, out_train, epochs = 20, verbose=1)

# Check on the test set
test_pred = mdl.predict(in_test) # test it
print(f"Model test prediction accuracy: {accuracy_score(np.argmax(out_test, axis=1), np.argmax(test_pred, axis=1))}")

Epoch 1/20
1750/1750 [==============================] - 4s 2ms/step - loss: 1.4233 - Accuracy: 0.6284
Epoch 2/20
1750/1750 [==============================] - 4s 2ms/step - loss: 0.5618 - Accuracy: 0.8322
Epoch 3/20
1750/1750 [==============================] - 4s 2ms/step - loss: 0.4554 - Accuracy: 0.8652
Epoch 4/20
1750/1750 [==============================] - 4s 2ms/step - loss: 0.4159 - Accuracy: 0.8764
Epoch 5/20
1750/1750 [==============================] - 4s 2ms/step - loss: 0.3925 - Accuracy: 0.8821
Epoch 6/20
1750/1750 [==============================] - 4s 2ms/step - loss: 0.3898 - Accuracy: 0.8842
Epoch 7/20
1750/1750 [==============================] - 4s 2ms/step - loss: 0.3806 - Accuracy: 0.8852
Epoch 8/20
1750/1750 [==============================] - 4s 2ms/step - loss: 0.3799 - Accuracy: 0.8858
Epoch 9/20
1750/1750 [==============================] - 4s 2ms/step - loss: 0.3760 - Accuracy: 0.8863
Epoch 10/20
1750/1750 [==============================] - 4s 2ms/step - loss: 0.372

### Export to TF Lite model

using the tf.lite.TFLinteConverter

In [5]:
converter = lite.TFLiteConverter.from_keras_model(mdl) # create the converter
tflite_model = converter.convert() # and run it

with open('model.tflite','wb') as f:
    f.write(tflite_model)


INFO:tensorflow:Assets written to: C:\Users\17204\AppData\Local\Temp\tmp1t1sqcnv\assets


INFO:tensorflow:Assets written to: C:\Users\17204\AppData\Local\Temp\tmp1t1sqcnv\assets
